In [1]:
# Import des librairies
import os
import pandas as pd
import numpy as np
import scipy.sparse as sparse
import implicit
from sklearn.preprocessing import OrdinalEncoder

In [2]:
# Import des données
clicks_sample = pd.read_csv('../1-Données/clicks_sample.csv', sep=",")
clicks_folder = "../1-Données/clicks/"
articles_embeddings = pd.read_pickle('../1-Données/articles_embeddings.pickle')

# Dataframe de tous les clics
df_clicks = pd.DataFrame()
for f in os.listdir(clicks_folder):
    df = pd.read_csv(clicks_folder + f)
    df_clicks = pd.concat([df_clicks, df])

In [3]:
# Dataframe comportant le nombre de clics par article par utilisateur
count_clics = (df_clicks[["user_id","click_article_id"]].reset_index()
                 .groupby(["user_id", "click_article_id"], as_index=False)
                 .agg(count_clics=("index", "count")))

In [6]:
count_clics.describe()

,count_clics
count,2.950710e+06
mean,1.012699e+00
std,1.344699e-01
min,1.000000e+00
25%,1.000000e+00
50%,1.000000e+00
75%,1.000000e+00
max,3.300000e+01


On observe une moyenne très faible mais toutefois supérieure à 1.
On peut donc utiliser le nombre d'articles cliqués comme "note" pour notre modèle de recommandations.

In [8]:
# Encodage des numéros d'articles et d'utilisateur entre 0 et leur nombre-1
encoder = OrdinalEncoder(dtype="int")
count_clics_encoded = pd.DataFrame(encoder.fit_transform(count_clics[["user_id", "click_article_id"]]),
                    columns=["user_id", "click_article_id"])
count_clics_encoded["count_clics"] = count_clics["count_clics"]

In [10]:
# Pré-traitement
# Création de la matrice creuse pour l'algorithme de recommandations
count_clics_sparsed = sparse.csr_matrix((count_clics_encoded["count_clics"].values,(count_clics_encoded["user_id"].values, count_clics_encoded["click_article_id"].values)),
                       shape=(count_clics_encoded["user_id"].max() + 1, count_clics_encoded["click_article_id"].max() + 1))

In [11]:
# Création du modèle
model = implicit.als.AlternatingLeastSquares(factors=50)

# Entraînement du modèle
model.fit(count_clics_sparsed)

C:\Users\luc19\anaconda3\lib\site-packages\implicit\utils.py:33: UserWarning: Intel MKL BLAS detected. Its highly recommend to set the environment variable 'export MKL_NUM_THREADS=1' to disable its internal multithreading
  warnings.warn(


  0%|          | 0/15 [00:00<?, ?it/s]

In [12]:
# Résultat pour un utilisateur
user_id = 1
recommendations = model.recommend(user_id, count_clics_sparsed[user_id], N=5)
recommendations

(array([37106, 27683, 17380, 38625, 18141]),
 array([0.46431664, 0.41406402, 0.14742061, 0.14593658, 0.13738927],
       dtype=float32))